In [10]:
import numpy as np
import pandas as pd
import torch
import random 
import os 

In [21]:
DATA_DIR = 'data/temp/'

In [54]:
def read_data() :
    IDATA_DIR = DATA_DIR + '1/'
    DDATA_DIR = DATA_DIR + '0/'

    idata = []
    ddata = []
    
    for f in os.listdir(IDATA_DIR):
        df = pd.read_csv(IDATA_DIR + f, header=None)
        idata.append(df.values)
    iLabels = [1 for i in range(len(idata))]

    for f in os.listdir(DDATA_DIR):
        df = pd.read_csv(DDATA_DIR + f, header=None)
        ddata.append(df.values)
    dLabels = [0 for i in range(len(ddata))]

    data = idata + ddata
    labels = iLabels + dLabels

    data = np.array(data)
    data = data.reshape(data.shape[:2])
    labels = np.array(labels)
    labels = labels.reshape(len(idata)+len(ddata),1)

    return data, labels

In [55]:
d,l = read_data()

In [57]:
d.shape

(200, 10)

In [58]:
l.shape

(200, 1)

In [59]:
class LSTM(torch.nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(LSTM,self).__init__()
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(input_size,hidden_size)
        self.linear = torch.nn.Linear(hidden_size,output_size)
        self.out = torch.nn.Sigmoid()
    def forward(self,x):
        x,_ = self.lstm(x)
        x = self.linear(x)
        x = self.out(x)
        return x
    def __call__(self,x):
        return self.forward(x)

In [60]:
lstm = LSTM(10,30,1)
optimizer = torch.optim.Adam(lstm.parameters(),lr=0.01)

In [63]:
def train(model:LSTM,optimizer,data,labels,epochs=10) :
    model.train()
    for epoch in range(1,epochs+1):
        x = torch.from_numpy(data).float()
        y = torch.from_numpy(labels).float()
        output = model(x)
        loss = torch.nn.functional.binary_cross_entropy(output,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'[+] Epoch : {epoch:<5d} :::   Loss : {loss.item():<10.5f}')

In [64]:
train(lstm,optimizer,d,l,epochs=100)

[+] Epoch : 10    :::   Loss : 0.00095   
[+] Epoch : 20    :::   Loss : 0.00083   
[+] Epoch : 30    :::   Loss : 0.00075   
[+] Epoch : 40    :::   Loss : 0.00069   
[+] Epoch : 50    :::   Loss : 0.00063   
[+] Epoch : 60    :::   Loss : 0.00058   
[+] Epoch : 70    :::   Loss : 0.00054   
[+] Epoch : 80    :::   Loss : 0.00050   
[+] Epoch : 90    :::   Loss : 0.00046   
[+] Epoch : 100   :::   Loss : 0.00043   


In [66]:
lstm.eval()

LSTM(
  (lstm): LSTM(10, 30)
  (linear): Linear(in_features=30, out_features=1, bias=True)
  (out): Sigmoid()
)

In [85]:
# Random Sample

sample = random.randint(0,len(d))
x = torch.from_numpy(d[sample]).float().unsqueeze(0)
y_true = l[sample]
y = lstm(x)
print(f'[+] Using Sample {sample}')
print(f'[+] Data : {x}')
print(f'[+] Prediction : {y.item():<10.5f}  :: True : {y_true.item():<10.5f}')

[+] Using Sample 59
[+] Data : tensor([[0.0383, 1.1800, 2.0042, 3.1711, 4.0389, 5.1854, 6.0814, 7.1778, 8.1069,
         9.0436]])
[+] Prediction : 0.99608     :: True : 1.00000   


In [92]:
# Decreasing Sample

array = [8,9,7,6,7,5,4,3,2,1]
y_true = np.array([0])
x = torch.from_numpy(np.array(array)).float().unsqueeze(0)
y = lstm(x)
print(f'[+] Using Sample {sample}')
print(f'[+] Data : {x}')
print(f'[+] Prediction : {y.item():<10.5f}  :: True : {y_true.item():<10.5f}')

[+] Using Sample 59
[+] Data : tensor([[8., 9., 7., 6., 7., 5., 4., 3., 2., 1.]])
[+] Prediction : 0.00197     :: True : 0.00000   


In [93]:
# Increasing Sample

array = [1,1,2,2,3,4,4,4,5,5]
y_true = np.array([1])
x = torch.from_numpy(np.array(array)).float().unsqueeze(0)
y = lstm(x)
print(f'[+] Using Sample {sample}')
print(f'[+] Data : {x}')
print(f'[+] Prediction : {y.item():<10.5f}  :: True : {y_true.item():<10.5f}')

[+] Using Sample 59
[+] Data : tensor([[1., 1., 2., 2., 3., 4., 4., 4., 5., 5.]])
[+] Prediction : 0.98603     :: True : 1.00000   


In [101]:
# Neutral Sample

array = np.ones(10)*1
y_true = np.array([1])
x = torch.from_numpy(np.array(array)).float().unsqueeze(0)
y = lstm(x)
print(f'[+] Using Sample {sample}')
print(f'[+] Data : {x}')
print(f'[+] Prediction : {y.item():<10.5f}  :: True : {y_true.item():<10.5f}')

[+] Using Sample 59
[+] Data : tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
[+] Prediction : 0.28460     :: True : 1.00000   
